In [1]:
import requests, time, ast, os
import pandas as pd
from neo4j import GraphDatabase
from graphdatascience import GraphDataScience
from ast import literal_eval
%env GOOGLE_APPLICATION_CREDENTIALS=/Users/mariusconjeaud/openstudybuildertests-9497bfb181cb.json
%load_ext dotenv
%dotenv


env: GOOGLE_APPLICATION_CREDENTIALS=/Users/mariusconjeaud/openstudybuildertests-9497bfb181cb.json


/Users/mariusconjeaud/.local/share/virtualenvs/ct-gov-osb-_azwkqY8/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bearer_token = !gcloud auth application-default print-access-token

# Load Data from CT.gov

In [44]:
studies = pd.read_csv("diabetes_type_1_all_ct_studies.csv")
studies.columns

Index(['Rank', 'NCT Number', 'Title', 'Acronym', 'Status', 'Study Results',
       'Conditions', 'Interventions', 'Outcome Measures',
       'Sponsor/Collaborators', 'Gender', 'Age', 'Phases', 'Enrollment',
       'Funded Bys', 'Study Type', 'Study Designs', 'Other IDs', 'Start Date',
       'Primary Completion Date', 'Completion Date', 'First Posted',
       'Results First Posted', 'Last Update Posted', 'Locations',
       'Study Documents', 'URL'],
      dtype='object')

# Call GCP Healthcare NLP API on each Study title

In [93]:
gcp_hc_nlp_url = "https://healthcare.googleapis.com/v1/projects/openstudybuildertests/locations/europe-west4/services/nlp:analyzeEntities"
headers = {
    "Authorization": "Bearer " + bearer_token[0],
    "Content-Type": "application/json",
}
data = {
    "nlpService": "projects/openstudybuildertests/locations/europe-west4/services/nlp",
    "documentContent": "Evaluation of the biomedical and psychosocial impact of automated Closed-Loop (Artificial Pancreas) insulin delivery in women with type 1 diabetes during pregnancy"
}

In [94]:
studies["nlp"] = ""

In [95]:

for i, study in studies.iterrows():
    if study["nlp"] == "":
        title = study["Title"]
        id = study["NCT Number"]
        data["documentContent"] = title
        response = requests.post(gcp_hc_nlp_url, headers=headers, json=data)
        if response.status_code == 200:
            studies.at[i, "nlp"] = response.json()
        else:
            time.sleep(60)

KeyboardInterrupt: 

In [96]:
studies.to_csv("studies_with_concepts.csv", index=False)

In [98]:
studies

,Rank,NCT Number,Title,Acronym,Status,Study Results,Conditions,Interventions,Outcome Measures,Sponsor/Collaborators,...,Start Date,Primary Completion Date,Completion Date,First Posted,Results First Posted,Last Update Posted,Locations,Study Documents,URL,nlp
0,1,NCT04291703,Low Dose Antithymocyte Globulin (ATG) to Delay...,TN28,Not yet recruiting,No Results Available,"Diabetes Mellitus, Type 1",Drug: Antithymocyte Globulin|Drug: Placebo (fo...,Progression to Stage 3 T1D,National Institute of Diabetes and Digestive a...,...,December 2022,"December 31, 2028","December 31, 2029","March 2, 2020",NaN,"November 28, 2022",NaN,NaN,https://ClinicalTrials.gov/show/NCT04291703,"{'entityMentions': [{'mentionId': '1', 'type':..."
1,2,NCT05593081,Multicenter Study of Fulminant Type 1 Diabetes...,NaN,Recruiting,No Results Available,"Diabetes Mellitus, Type 1",Genetic: HLA,Change in serum hemoglobin A1c level|Change in...,Second Xiangya Hospital of Central South Unive...,...,"May 20, 2022","December 31, 2027","December 31, 2027","October 25, 2022",NaN,"October 25, 2022","Quanzhou First People's Hospital, Quanzhou, Fu...",NaN,https://ClinicalTrials.gov/show/NCT05593081,"{'entityMentions': [{'mentionId': '1', 'type':..."
2,3,NCT04616391,AHCL System Initiation in T1D Patients naïve t...,NaN,Unknown status,No Results Available,"Diabetes Mellitus, Type 1",Device: insulin pump Medtronic 780G,Between group TIR difference|Between group TIR...,Jagiellonian University|Medtronic Poland Spółk...,...,"November 2, 2020","March 31, 2021","July 1, 2021","November 5, 2020",NaN,"November 5, 2020",NaN,NaN,https://ClinicalTrials.gov/show/NCT04616391,{}
3,4,NCT04670198,A Psychosocial Education Programme for Young P...,YES,"Active, not recruiting",No Results Available,"Diabetes Mellitus, Type 1",Behavioral: Youth Empowerment Skills|Behaviora...,Glycated haemoglobin (HbA1c)|Frequency of bloo...,King's College London|Guy's and St Thomas' Fou...,...,"June 1, 2021","March 31, 2023","March 31, 2023","December 17, 2020",NaN,"April 7, 2022","Guy's and St Thomas' NHS Foundation Trust, Lon...",NaN,https://ClinicalTrials.gov/show/NCT04670198,"{'entityMentions': [{'mentionId': '1', 'type':..."
4,5,NCT02215200,ATG-GCSF in New Onset Type 1 Diabetes,ATG-GCSF,Completed,Has Results,"Diabetes Mellitus, Type 1",Drug: Anti-Thymocyte Globulin (ATG)|Drug: Gran...,Change in Area Under the Stimulated C-peptide ...,National Institute of Diabetes and Digestive a...,...,December 2014,August 2017,August 2018,"August 13, 2014","March 2, 2020","March 2, 2020","University of California - San Francisco, San ...","""Study Protocol"", https://ClinicalTrials.gov/P...",https://ClinicalTrials.gov/show/NCT02215200,"{'entityMentions': [{'mentionId': '1', 'type':..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3132,3133,NCT00246844,Assessment of Insulin Production From Native P...,NaN,Completed,No Results Available,Long-Functioning Pancreas Allografts|Pancreas ...,NaN,NaN,National Institute of Diabetes and Digestive a...,...,"October 26, 2005",NaN,"August 15, 2007","October 30, 2005",NaN,"July 2, 2017","National Institutes of Health Clinical Center,...",NaN,https://ClinicalTrials.gov/show/NCT00246844,
3133,3134,NCT04419779,Evaluation of the Efficacy and Safety of Duode...,REVITALIZE 1,Recruiting,No Results Available,Type 2 Diabetes,Device: Duodenal Mucosal Resurfacing (DMR)|Dev...,The primary endpoint is the percentage of subj...,"Fractyl Health, Inc.|Fractyl Laboratories, Inc.",...,"March 8, 2021",June 2023,January 2025,"June 5, 2020",NaN,"October 17, 2022","Mayo Clinic Arizona, Scottsdale, Arizona, Unit...",NaN,https://ClinicalTrials.gov/show/NCT04419779,
3134,3135,NCT05132179,"Treating Diabetic Foot Ulcers, Comparing Two T...",NaN,Recruiting,No Results Available,Diabetic Foot Ulcer,"Drug: Dakins Full Strength Solution, 0.5% Topi...",Time to healing|Proportion of DFU that heals w...,Vastra Gotaland Region,...,"February 1, 2022","December 31, 2023","December 

In [92]:
response = requests.post(gcp_hc_nlp_url, headers=headers, json=data)
response.json()

{'error': {'code': 403,
  'message': 'SNOMEDCT_US vocabulary is not accessible from user location',
  'status': 'PERMISSION_DENIED'}}

In [99]:
print(len(studies[studies['nlp'] == '']))

1836


# Prepare and inject data

In [62]:
studies_concept = pd.read_csv("studies_with_concepts.csv")
studies_concept_no_na = studies_concept[studies_concept.nlp.notnull()]
studies_concept_no_na = studies_concept_no_na[['NCT Number', 'Title', 'Interventions', 'Outcome Measures', 'Age', 'Study Designs', 'nlp']]
studies_concept_no_na["title_embedding"] = ""
studies_concept_no_na["interventions_embedding"] = ""
studies_concept_no_na["outcome_embedding"] = ""
studies_concept_no_na["designs_embedding"] = ""
studies_concept_no_na['Outcome Measures'] = studies_concept_no_na['Outcome Measures'].str.replace('\|',', ', regex=True)
studies_concept_no_na = studies_concept_no_na.fillna('')

open_ai_url = "https://api.openai.com/v1/embeddings"
headers = {
    "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    "Content-Type": "application/json",
}
data = {
    "model": "text-embedding-ada-002",
    "input": ""
}

studies_concept_no_na.head(5)

,NCT Number,Title,Interventions,Outcome Measures,Age,Study Designs,nlp,title_embedding,interventions_embedding,outcome_embedding,designs_embedding
0,NCT04291703,Low Dose Antithymocyte Globulin (ATG) to Delay...,"Drug: Antithymocyte Globulin, Drug: Placebo (f...",Progression to Stage 3 T1D,"12 Years to 34 Years (Child, Adult)","Allocation: Randomized, Intervention Model: Pa...","{'entityMentions': [{'mentionId': '1', 'type':...",,,,
1,NCT05593081,Multicenter Study of Fulminant Type 1 Diabetes...,Genetic: HLA,"Change in serum hemoglobin A1c level, Change i...","Child, Adult, Older Adult","Observational Model: Other, Time Perspective: ...","{'entityMentions': [{'mentionId': '1', 'type':...",,,,
2,NCT04616391,AHCL System Initiation in T1D Patients naïve t...,Device: insulin pump Medtronic 780G,"Between group TIR difference, Between group TI...",26 Years to 60 Years (Adult),"Allocation: Randomized, Intervention Model: Pa...",{},,,,
3,NCT04670198,A Psychosocial Education Programme for Young P...,"Behavioral: Youth Empowerment Skills, Behavior...","Glycated haemoglobin (HbA1c), Frequency of blo...","14 Years to 19 Years (Child, Adult)","Allocation: Randomized, Intervention Model: Cr...","{'entityMentions': [{'mentionId': '1', 'type':...",,,,
4,NCT02215200,ATG-GCSF in New Onset Type 1 Diabetes,"Drug: Anti-Thymocyte Globulin (ATG), Drug: Gra...",Change in Area Under the Stimulated C-peptide ...,"12 Years to 45 Years (Child, Adult)","Allocation: Randomized, Intervention Model: Pa...","{'entityMentions': [{'mentionId': '1', 'type':...",,,,


In [ ]:
# for i, study in studies_concept_no_na.iterrows():
#     if study["title_embedding"] == "":
#         title = study["Title"]
#         data["input"] = title
#         response = requests.post(open_ai_url, headers=headers, json=data)
#         if response.status_code == 200:
#             try:
#                 studies_concept_no_na.at[i, "title_embedding"] = response.json()['data'][0]['embedding']
#             except:
#                 continue
#         else:
#             time.sleep(60)
#     if study["interventions_embedding"] == "":
#         interventions = study["Interventions"]
#         data["input"] = interventions
#         response = requests.post(open_ai_url, headers=headers, json=data)
#         if response.status_code == 200:
#             try:
#                 studies_concept_no_na.at[i, "interventions_embedding"] = response.json()['data'][0]['embedding']
#             except:
#                 continue
#         else:
#             time.sleep(60)
#     if study["outcome_embedding"] == "":
#         outcome = study["Outcome Measures"]
#         data["input"] = outcome
#         response = requests.post(open_ai_url, headers=headers, json=data)
#         if response.status_code == 200:
#             try:
#                 studies_concept_no_na.at[i, "outcome_embedding"] = response.json()['data'][0]['embedding']
#             except:
#                 continue
#         else:
#             time.sleep(60)
#     if study["designs_embedding"] == "":
#         designs = study["Study Designs"]
#         data["input"] = designs
#         response = requests.post(open_ai_url, headers=headers, json=data)
#         if response.status_code == 200:
#             try:
#                 studies_concept_no_na.at[i, "designs_embedding"] = response.json()['data'][0]['embedding']
#             except:
#                 continue
#         else:
#             time.sleep(60)

# studies_concept_no_na

In [122]:
studies_concept_no_na = studies_concept_no_na[studies_concept_no_na['title_embedding'] != '']
studies_concept_no_na = studies_concept_no_na[studies_concept_no_na['outcome_embedding'] != '']
print(len(studies_concept_no_na))
studies_concept_no_na.to_csv("studies_with_concepts_and_embeddings.csv", index=False)

1281


In [6]:
studies_concept_no_na = pd.read_csv("studies_with_concepts_and_embeddings.csv")
studies_concept_no_na

,NCT Number,Title,Interventions,Outcome Measures,Age,Study Designs,nlp,title_embedding,interventions_embedding,outcome_embedding,designs_embedding
0,NCT04291703,Low Dose Antithymocyte Globulin (ATG) to Delay...,Drug: Antithymocyte Globulin|Drug: Placebo (fo...,Progression to Stage 3 T1D,"12 Years to 34 Years (Child, Adult)",Allocation: Randomized|Intervention Model: Par...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.024888411, -0.021991916, 0.007529549, -0.0...","[-0.04231575, -0.0023409699, 0.0062007634, -0....","[-0.00057642633, -0.030631753, 0.02350654, -0....","[-0.024420716, -0.0107903965, 0.0046459557, -0..."
1,NCT05593081,Multicenter Study of Fulminant Type 1 Diabetes...,Genetic: HLA,"Change in serum hemoglobin A1c level, Change i...","Child, Adult, Older Adult",Observational Model: Other|Time Perspective: O...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.005025614, -0.011295795, 0.019006697, -0.0...","[-0.018663272, 0.018031064, 0.022983352, -0.02...","[-0.004576705, -0.02505552, 0.026270803, -0.03...","[-0.012568679, -0.021009931, 0.017122163, -0.0..."
2,NCT04616391,AHCL System Initiation in T1D Patients naïve t...,Device: insulin pump Medtronic 780G,"Between group TIR difference, Between group TI...",26 Years to 60 Years (Adult),Allocation: Randomized|Intervention Model: Par...,{},"[0.008752234, 0.010164336, 0.017112995, -0.006...","[-0.01514027, -0.00065040245, 0.015260114, -0....","[-0.018643277, -0.009801525, 0.025900744, -0.0...","[-0.012270932, -0.0050068027, 0.005177415, -0...."
3,NCT04670198,A Psychosocial Education Programme for Young P...,Behavioral: Youth Empowerment Skills|Behaviora...,"Glycated haemoglobin (HbA1c), Frequency of blo...","14 Years to 19 Years (Child, Adult)",Allocation: Randomized|Intervention Model: Cro...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.01474828, -0.026422344, 0.022248304, -0.04...","[-0.0399825, -0.009807283, 0.026906125, -0.022...","[-0.004271557, -0.004638721, 0.02323622, -0.05...","[-0.016580248, 0.001029644, 0.0015577091, -0.0..."
4,NCT02215200,ATG-GCSF in New Onset Type 1 Diabetes,Drug: Anti-Thymocyte Globulin (ATG)|Drug: Gran...,Change in Area Under the Stimulated C-peptide ...,"12 Years to 45 Years (Child, Adult)",Allocation: Randomized|Intervention Model: Par...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.019683944, -0.006886067, 0.0051496383, -0....","[-0.032275654, 0.003974091, 0.014581728, -0.01...","[-0.024471877, -0.0057166833, 0.007196837, -0....","[-0.029715126, -0.012106889, 0.005187733, -0.0..."
...,...,...,...,...,...,...,...,...,...,...,...
1276,NCT00837759,Novel Therapy to Preserve Beta Cell Function i...,Drug: Insulin|Drug: Lansoprazole|Drug: Sitagli...,"Change in C-peptide, Glycemia Control (Change ...","16 Years to 30 Years (Child, Adult)",Allocation: N/A|Intervention Model: Single Gro...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.007807395, -0.021682782, 0.024749974, 0.00...","[-0.01583245, 0.008632265, 0.045269668, -0.003...","[-0.028385984, -0.0058840127, 0.033143412, -0....","[-0.014651199, -0.00798977, 0.012112885, -0.00..."
1277,NCT05537233,ADJUnct Semaglutide Treatment in Type 1 Diabetes,Drug: Semaglutide|Drug: Placebo,Proportion of adults with T1D achieving compos...,18 Years to 60 Years (Adult),Allocation: Randomized|Intervention Model: Par...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.021696474, -0.016885517, 0.012121727, 0.01...","[-0.03955424, -0.007101003, 0.0280341, 0.01846...","[-0.021591319, -0.016848484, 0.027261224, -0.0...","[-0.03131989, -0.008988756, 0.0040819296, -0.0..."
1278,NCT04149262,Fiasp® Versus NovoRapid® in Children With Type...,Drug: Fiasp|Drug: Novorapid|Device: 640G pump,"1-hour glucose levels on Fiasp, 1-hour glucose...","2 Years to 18 Years (Child, Adult)",Allocation: Randomized|Intervention Model: Cro...,"{'entityMentions': [{'mentionId': '1', 'type':...","[-0.01590547, 0.013424547, 0.0034353884, -0.01...","[-0.030471023, 0.007462567, 0.02

In [7]:
def parse_piped_cell(cell):
    output = {}
    if not pd.isna(cell):
        _split = cell.split('|')
        for el in _split:
            parsed_el = el.split(': ')
            if parsed_el[0] in output:
                output[parsed_el[0]].append(parsed_el[1])
            else:
                output[parsed_el[0]] = [parsed_el[1]]
    return output

def parse_nlp_entities(cell):
    output = []
    exclusion_list = ["Diabetes Mellitus, Insulin-Dependent"]
    if "entities" in ast.literal_eval(cell):
        entities = ast.literal_eval(cell)["entities"]
        output = [entity["preferredTerm"] for entity in entities if entity["preferredTerm"] not in exclusion_list]
    return output

def create_study(tx, study_id, study_props):
    result = tx.run(
        """
        MERGE (s:Study {id: $study_id})
        SET s+=$study_props
        """,
        study_id=study_id,
        study_props=study_props)
    summary = result.consume()
    return summary

def link_study_interventions(tx, study_id, interventions):
    for intervention_type, intervention_names in interventions.items():
        for name in intervention_names:
            _ = tx.run(
                f"""
                MERGE (i:Intervention:{intervention_type.replace(' ', '')} {{name: $name}})
                WITH i
                MATCH (s:Study {{id: $study_id}})
                MERGE (s)-[:HAS_INTERVENTION]->(i)
                """,
                study_id=study_id,
                name=name
            )

def link_study_designs(tx, study_id, designs):
    for design_type, design_names in designs.items():
        for name in design_names:
            _ = tx.run(
                f"""
                MERGE (i:Design:{design_type.replace(' ', '')} {{name: $name}})
                WITH i
                MATCH (s:Study {{id: $study_id}})
                MERGE (s)-[:HAS_DESIGN]->(i)
                """,
                study_id=study_id,
                name=name
            )

def link_study_entities(tx, study_id, entities):
    for entity in entities:
        _ = tx.run(
            """
            MERGE (e:Entity {preferred_term: $entity})
            WITH e
            MATCH (s:Study {id: $study_id})
            MERGE (s)-[:HAS_ENTITY]->(e)
            """,
            study_id=study_id,
            entity=entity
        )

In [4]:
# uri = os.getenv("DATABASE_URI_AURA")
uri = "bolt://localhost:7687"
user = "neo4j"
password = os.getenv("DATABASE_PASSWORD")
# database="neo4j"
database = "ct.gov"
driver = GraphDatabase.driver(uri, auth=(user, password))

# nodes_created = 0
# elapsed_time = 0
# with driver.session(database=database) as session:
#     for index, row in studies_concept_no_na.iterrows():
#         _interventions = parse_piped_cell(row["Interventions"])
#         _designs = parse_piped_cell(row["Study Designs"])
#         _entities = parse_nlp_entities(row["nlp"])
#         study_props = { 
#             "title": row["Title"], 
#             "title_embedding": literal_eval(row["title_embedding"]),
#             "outcome": row["Outcome Measures"],
#             "outcome_embedding": literal_eval(row["outcome_embedding"]),
#         }
#         summary = session.execute_write(create_study, study_id=row['NCT Number'], study_props=study_props)

#         session.execute_write(link_study_interventions, study_id=row['NCT Number'], interventions=_interventions)
#         session.execute_write(link_study_designs, study_id=row['NCT Number'], designs=_designs)
#         if _entities != []:
#             session.execute_write(link_study_entities, study_id=row['NCT Number'], entities=_entities)

#         nodes_created+=summary.counters.nodes_created
#         elapsed_time+=summary.result_available_after
        
# print(f"Created {nodes_created} nodes in {elapsed_time} ms.")

In [ ]:
# studies_concept_no_na['Interventions'] = studies_concept_no_na['NCT Number'].map(studies_concept.set_index('NCT Number')['Interventions'])
# studies_concept_no_na['Study Designs'] = studies_concept_no_na['NCT Number'].map(studies_concept.set_index('NCT Number')['Study Designs'])
# studies_concept_no_na.head()

In [5]:
gds = GraphDataScience(uri, auth=(user, password))
gds.set_database(database)

# gds.run_cypher(
# """
#     MATCH (n:Study), (m:Study)
#     WHERE id(n) < id(m)
#     MATCH (n)--(shared)--(m)
#     WHERE NOT shared:Study AND NOT shared:Design
#     WITH n, m, count(DISTINCT shared) AS weight
#     MERGE (n)-[rel:SHARES]->(m)
#         SET rel.weight=weight
# """)

embeddings_projection, result = gds.graph.project(
    "embeddings",
    ["Study"],
    ["SHARES"],
    nodeProperties=["title_embedding", "outcome_embedding"],
    relationshipProperties=["weight"]
    )
embeddings_projection

Loading: 100%|██████████| 100.0/100 [00:00<00:00, 149.03%/s]


Graph({'graphName': 'embeddings', 'nodeCount': 1281, 'relationshipCount': 26394, 'database': 'ct.gov', 'configuration': {}, 'schema': {'graphProperties': {}, 'relationships': {'SHARES': {'weight': 'Float (DefaultValue(NaN), PERSISTENT, Aggregation.NONE)'}}, 'nodes': {'Study': {'outcome_embedding': 'List of Float (DefaultValue(null), PERSISTENT)', 'title_embedding': 'List of Float (DefaultValue(null), PERSISTENT)'}}}, 'memoryUsage': '34 MiB'})

## Use graph embeddings

In [15]:
result = gds.fastRP.mutate.estimate(
    embeddings_projection,
    mutateProperty="fastrp",
    randomSeed=42,
    embeddingDimension=128,
    relationshipWeightProperty="weight",
    iterationWeights=[0, 1, 1],
    propertyRatio=1,
    featureProperties=["title_embedding", "outcome_embedding"],
)

print(f"Required memory for running FastRP: {result['requiredMemory']}")

Required memory for running FastRP: 2042 KiB


In [16]:
result = gds.fastRP.mutate(
    embeddings_projection,
    mutateProperty="fastrp",
    randomSeed=42,
    embeddingDimension=128,
    relationshipWeightProperty="weight",
    iterationWeights=[0, 1, 1],
    propertyRatio=1,
    featureProperties=["title_embedding", "outcome_embedding"],
)
print(f"Number of embedding vectors produced: {result['nodePropertiesWritten']}")

Number of embedding vectors produced: 1281


In [17]:
result = gds.knn.mutate(
    embeddings_projection,
    topK=3,
    nodeLabels=["Study"],
    nodeProperties=["fastrp"],
    randomSeed=42,
    concurrency=1,
    sampleRate=1.0,
    similarityCutoff=0.5,
    deltaThreshold=0.0,
    # writeRelationshipType="SIMILAR",
    # writeProperty="score",
    mutateRelationshipType="SIMILAR",
    mutateProperty="score",
)

print(f"Relationships produced: {result['relationshipsWritten']}")
print(f"Nodes compared: {result['nodesCompared']}")
print(f"Mean similarity: {result['similarityDistribution']['mean']}")

Relationships produced: 2397
Nodes compared: 1281
Mean similarity: 0.9988221831753397


In [18]:
# Evaluate clustering quality

result = gds.wcc.stats(embeddings_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR"])
print(f"Component count: {result['componentCount']}")
print(f"Component distribution: {result['componentDistribution']}")

Component count: 488
Component distribution: {'p99': 4, 'min': 1, 'max': 757, 'mean': 2.625, 'p90': 1, 'p50': 1, 'p999': 757, 'p95': 1, 'p75': 1}


In [19]:
gds.graph.relationship.write(embeddings_projection, 'SIMILAR', 'score')

writeMillis                     74
graphName               embeddings
relationshipType           SIMILAR
relationshipProperty         score
relationshipsWritten          2397
propertiesWritten             2397
Name: 0, dtype: object

## Direct similarity on properties

In [34]:
result_direct = gds.knn.mutate(
    embeddings_projection,
    topK=3,
    nodeLabels=["Study"],
    nodeProperties=["title_embedding", "outcome_embedding"],
    randomSeed=42,
    concurrency=1,
    sampleRate=1.0,
    similarityCutoff=0.9,
    deltaThreshold=0.0,
    mutateRelationshipType="SIMILAR_DIRECT",
    mutateProperty="score",
)

print(f"Relationships produced: {result_direct['relationshipsWritten']}")
print(f"Nodes compared: {result_direct['nodesCompared']}")
print(f"Mean similarity: {result_direct['similarityDistribution']['mean']}")

Knn: 100%|██████████| 100.0/100 [00:04<00:00, 21.45%/s]

Relationships produced: 3842
Nodes compared: 1281
Mean similarity: 0.9483834434958065


In [35]:
# Evaluate clustering quality

result = gds.wcc.stats(embeddings_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR_DIRECT"])
print(f"Component count: {result['componentCount']}")
print(f"Component distribution: {result['componentDistribution']}")

result = gds.louvain.stats(embeddings_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR_DIRECT"])
print(f"Community count: {result['communityCount']}")
print(f"Community distribution: {result['communityDistribution']}")

Component count: 2
Component distribution: {'p99': 1264, 'min': 17, 'max': 1264, 'mean': 640.5, 'p90': 1264, 'p50': 17, 'p999': 1264, 'p95': 1264, 'p75': 1264}


Louvain: 100%|██████████| 100.0/100 [00:00<00:00, 287.04%/s]

Community count: 20
Community distribution: {'p99': 191, 'min': 9, 'max': 191, 'mean': 64.05, 'p90': 124, 'p50': 44, 'p999': 191, 'p95': 169, 'p75': 89}


In [36]:
gds.graph.relationship.write(embeddings_projection, 'SIMILAR_DIRECT', 'score')

writeMillis                         78
graphName                   embeddings
relationshipType        SIMILAR_DIRECT
relationshipProperty             score
relationshipsWritten              3842
propertiesWritten                 3842
Name: 0, dtype: object

## Louvain and PageRank

In [6]:
similarity_projection, result = gds.graph.project(
    "similarity",
    ["Study"],
    ["SIMILAR_DIRECT"],
    relationshipProperties=["score"]
    )
similarity_projection

Graph({'graphName': 'similarity', 'nodeCount': 1281, 'relationshipCount': 3842, 'database': 'ct.gov', 'configuration': {}, 'schema': {'graphProperties': {}, 'relationships': {'SIMILAR_DIRECT': {'score': 'Float (DefaultValue(NaN), PERSISTENT, Aggregation.NONE)'}}, 'nodes': {'Study': {}}}, 'memoryUsage': '4687 KiB'})

In [12]:
result = gds.louvain.stats(similarity_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR_DIRECT"], relationshipWeightProperty="score")
print(f"Community count: {result['communityCount']}")
print(f"Community distribution: {result['communityDistribution']}")

result = gds.pageRank.stats(similarity_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR_DIRECT"], relationshipWeightProperty="score")
print(f"Centrality distribution: {result['centralityDistribution']}")

Louvain: 100%|██████████| 100.0/100 [00:00<00:00, 127.43%/s]

Community count: 22
Community distribution: {'p99': 145, 'min': 9, 'max': 145, 'mean': 58.22727272727273, 'p90': 111, 'p50': 61, 'p999': 145, 'p95': 131, 'p75': 78}
Centrality distribution: {'p99': 8.657042503356934, 'min': 0.14999961853027344, 'max': 13.867918968200684, 'mean': 0.9612402547438753, 'p90': 2.464003562927246, 'p50': 0.36838436126708984, 'p999': 11.799315452575684, 'p95': 3.7859792709350586, 'p75': 0.9656782150268555}


In [13]:
result = gds.louvain.write(similarity_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR_DIRECT"], relationshipWeightProperty="score", writeProperty="louvain")
result = gds.pageRank.write(similarity_projection, nodeLabels=["Study"], relationshipTypes=["SIMILAR_DIRECT"], relationshipWeightProperty="score", writeProperty="pagerank")

Louvain: 100%|██████████| 100.0/100 [00:00<00:00, 295.45%/s]


## Utils

In [33]:
# Clean up

# gds.graph.drop(embeddings_projection)
# gds.run_cypher("MATCH ()-[rel:SIMILAR]-() DELETE rel")
# gds.run_cypher("MATCH ()-[rel:SIMILAR_DIRECT]-() DELETE rel")
# gds.run_cypher("MATCH ()-[rel:SHARES]-() DELETE rel")
gds.graph.relationships.drop(embeddings_projection, 'SIMILAR_DIRECT')

graphName                    embeddings
relationshipType         SIMILAR_DIRECT
deletedRelationships               3842
deletedProperties       {'score': 3842}
Name: 0, dtype: object

# Recap

First tries gave me tremendously high similarities => nodes were sharing too many things (design)

=> Reduced scope by removing Entities => But then items which text were different were linked together ; and some sharing elements were not

=> Also tried with propertyRatio 0.9 and limited to direct neighbours

=> Added Entities again, but set FastRP propertyRatio to 1 and extended to 2nd and 3rd level neighbours

=> *It finally creates nice communities of items that share some keywords while excluding too generic keywords !*